In [1]:
import tweepy
import pandas as pd

### Set up the API tokens


In [2]:
import secret

auth = tweepy.OAuthHandler(secret.consumer_key, secret.consumer_secret)
auth.set_access_token(secret.access_token, secret.access_token_secret)

api = tweepy.API(auth)

# If the authentication was successful, this should print the
# screen name / username of the account
print(api.verify_credentials().screen_name)

dimitrioshal


### Use a list of discovered channels

In [137]:
filtered_agg_tweet = pd.read_csv('/Users/dimmehal/PycharmProjects/twitter_sentiment/data/news_channels_3.csv')
filtered_agg_tweet

,Unnamed: 0,user_name,user_description,date,user_followers
0,0,#All Bills Paid,Help Bitcoin Change Lives through The Lightnin...,675,77601.0
1,1,#All Bills Paid,Start Receiving Bitcoin with https://t.co/bK2f...,334,77633.0
2,2,#Bitcoin,#Bitcoin,177,11531.0
3,3,#Help Me Pay My Bills,https://t.co/vBGrS2H9me\n\nPutting the Lightni...,3451,77395.0
4,4,#Help Me Pay My Bills,https://t.co/vBGrS2H9me\n\nPutting the Lightni...,2125,77516.0
...,...,...,...,...,...
236,236,☣surferjim is more stoked than you- regularly.🟩,NAP. ₿itcoin not blockchain. Lifelong surfer &...,93,17067.0
237,237,𓃓 YearoftheBULL 𓄀 Φ 1.618 Φ Golden Ratio Φ,Fascinated by patterns and symbols.\nFeatured ...,162,14247.0
238,238,🇺🇲🇮🇹3ℓℓ3🇵🇹🇪🇺,"🇺🇲🌊 in EU $poodl proud, #animalrights #Dogsof...",59,17409.0
239,239,🌐 DeFiNewsLead 🌐,#DeFi News | #SOLANA #KCC #BINANCE #ETHEREUM |...,215,11635.0


### Look up the names and try to find screen-names and user_ids
As the dataset only contained the name (ie Dimitrios Chalatsis) and not the screen_name (i.e. dimitrioshal) or the user_id. This appears to be a problem as the name can be changed quite often and is hard to trace back

In [138]:
def get_highest_following(row):
    users = api.search_users(row['user_name'])
    if not users:
        return None
    users = sorted(users, key=lambda u:u._json['followers_count'], reverse=True)
    
    # if the user followers are not consistent
    follower_count = users[0]._json['followers_count']
#     if (abs(follower_count - row['user_followers'])/row['user_followers']) > 0.2:

    return users[0]

filtered_agg_tweet['user_object'] = filtered_agg_tweet.apply(get_highest_following, axis=1)

In [144]:
filtered_agg_tweet

,Unnamed: 0,user_name,user_description,date,user_followers,user_object
0,0,#All Bills Paid,Help Bitcoin Change Lives through The Lightnin...,675,77601.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
1,1,#All Bills Paid,Start Receiving Bitcoin with https://t.co/bK2f...,334,77633.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
2,2,#Bitcoin,#Bitcoin,177,11531.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
3,3,#Help Me Pay My Bills,https://t.co/vBGrS2H9me\n\nPutting the Lightni...,3451,77395.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
4,4,#Help Me Pay My Bills,https://t.co/vBGrS2H9me\n\nPutting the Lightni...,2125,77516.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
...,...,...,...,...,...,...
236,236,☣surferjim is more stoked than you- regularly.🟩,NAP. ₿itcoin not blockchain. Lifelong surfer &...,93,17067.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
237,237,𓃓 YearoftheBULL 𓄀 Φ 1.618 Φ Golden Ratio Φ,Fascinated by patterns and symbols.\nFeatured ...,162,14247.0,None
238,238,🇺🇲🇮🇹3ℓℓ3🇵🇹🇪🇺,"🇺🇲🌊 in EU $poodl proud, #animalrights #Dogsof...",59,17409.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...
239,239,🌐 DeFiNewsLead 🌐,#DeFi News | #SOLANA #KCC #BINANCE #ETHEREUM |...,215,11635.0,User(_api=<tweepy.api.API object at 0x7f9cdd76...


In [148]:
tweet_accounts = filtered_agg_tweet[~filtered_agg_tweet['user_object'].isnull()]

tweet_accounts['discovered_followers'] = tweet_accounts['user_object'].apply(lambda x: x._json['followers_count'])



/var/folders/m1/8wsbp1px1rn5_gc72hrvpkbr0000gn/T/ipykernel_27426/1774602726.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_accounts['discovered_followers'] = tweet_accounts['user_object'].apply(lambda x: x._json['followers_count'])


In [153]:
def filter_wrong_accounts(row):
    if row['discovered_followers'] - row['user_followers'] < 0.1*row['discovered_followers']:
        return False
    return True

ta_filtered = tweet_accounts[tweet_accounts.apply(filter_wrong_accounts,axis=1)]

In [166]:
user_ids = ta_filtered.apply(lambda row: row.user_object._json['id'], axis=1).values
user_ids

array([          357312062,           934026494,            32137602,
       1151153768165076992, 1173425402523652096, 1158405937612197888,
        958118843636854784, 1380537830779260936,          3185716686,
       1266954523081183232,           233498480,  883390115111985154,
                 124111156, 1021643080208801792,  943506806931734528,
       1240876822981685249,  970994516357472257,           361289499,
                3367334171,           845804107, 1163859351087591424,
       1163859351087591424,           357312062,            14508177,
                  24026381,            24026381,            67109577,
                 241664456,           241664456, 1373334050048466946,
                 129935623,          1333467482,          2260491445,
       1258075532945588224,  734390196771381248,          2207129125,
                 219890533,  798107102732500992, 1328156809341779968,
        952978915101831168,  853931836438740992, 1172632615393865730,
       1353384573435

In [39]:

found_users = []
# fetching the user
for name in names:
    try :
        user = api.get_user(id=name)
        found_users.append(user._json['id'])
    except:
        pass 
        

In [225]:
tweet_users = user_ids[102:]
tweet_users

array([         1572616332,            14136886,  953559767242498049,
                  14321959,          1959586644, 1142737563481980931,
        955920554325872640, 1039833297751302144,  968796006576947200,
                  14101605,           182631148,  856530759116402690,
                 303862998, 1237579017198415872, 1371741609449037844,
                3383733773, 1426278754960150537,  940960926890647553,
       1178503185545084929, 1178503185545084929, 1022821051187822593,
                3385236489, 1012626486996750336, 1309703843949682694,
       1130997943153111040, 1025967921472786432])

In [223]:
import numpy as np
np.where(user_ids == 1572616332)

(array([102]),)

### Pull the tweets here

In [226]:
def get_tweets(user_id, max_id):
    if max_id == None:
        api_tweets = api.user_timeline(user_id=user_id, count=1000)
    else:
        api_tweets = api.user_timeline(user_id=user_id, count=1000, max_id=max_id)
    api_tweets_1 = [api_tweets[i]._json  for i in range(len(api_tweets))]
    res = pd.DataFrame.from_dict(api_tweets_1)
    #res.drop('user', axis=1)
    if res.empty:
        return (res, None)
    return (res, res.iloc[-1].id)
 # Bitcoin
user_list = tweet_users
#all_tweets = None
for user_id in user_list:


    tweets_account, max_id = get_tweets(user_id, None)

    for i in range(1000):
        res, last_id = get_tweets(user_id, max_id)
        tweets_account = pd.concat([tweets_account, res])
        if max_id == last_id:
            break
        max_id = last_id
    
    #all_tweets.reset_index()
#     print(all_tweets)
#     print(tweets_account)
    if all_tweets is None:
        all_tweets = tweets_account
    else:
        all_tweets = all_tweets.append(tweets_account)

all_tweets = all_tweets.reset_index()
all_tweets.to_json('/Users/dimmehal/PycharmProjects/twitter_sentiment/data/bitcoin_tweets_2.json')

ValueError: cannot insert level_0, already exists

In [230]:
all_tweets = all_tweets.reset_index()
all_tweets.to_json('/Users/dimmehal/PycharmProjects/twitter_sentiment/data/bitcoin_tweets_2.json')

In [229]:
all_tweets = all_tweets.drop(columns=['level_0', 'index'])

In [219]:
all_tweets.iloc[-1].user

{'id': 1572616332,
 'id_str': '1572616332',
 'name': 'TradersApprentice',
 'screen_name': 'Trader4lyf',
 'location': '',
 'description': 'former ibanker & founder 150K strong Traders Apprentice (TAP) 🇵🇭 🇺🇸🇸🇬🇦🇪investing group, Bitcoin advocate, Wharton MBA🇺🇸, global macro poker strategist',
 'url': None,
 'entities': {'description': {'urls': []}},
 'protected': False,
 'followers_count': 26059,
 'friends_count': 733,
 'listed_count': 174,
 'created_at': 'Sat Jul 06 12:03:10 +0000 2013',
 'favourites_count': 43936,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 39526,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_image

In [183]:
all_tweet = pd.read_json(path_or_buf='/Users/dimmehal/PycharmProjects/twitter_sentiment/data/bitcoin_twet.json')

ValueError: Expected object or value

In [140]:

all_tweets = all_tweets.reset_index()

In [141]:
all_tweets.to_json('/Users/dimmehal/PycharmProjects/twitter_sentiment/data/bitcoin_tw.json')


In [143]:

all_tweets[all_tweets['id'] == 1007133581603270657]

,index,created_at,id,id_str,text,truncated,entities,extended_entities,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
3256,65,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3257,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3258,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3259,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3260,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3337,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3338,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
3339,0,Thu Jun 14 05:32:12 +0000 2018,1007133581603270657,1007133581603270657,It's pretty ironic that a bunch of people prom...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,...,89,256,False,False,False,en,NaN,NaN,NaN,NaN
